# Collecting summoner IDs per Region, Division & Tier

In [ ]:
## Notes

! Display line numbers by clicking left of the green bar of the cell and pressing L. Has to be done on a per cell basis. 

Source: https://riot-api-libraries.readthedocs.io/en/latest/collectingdata.html

You can also use the League endpoints to get lists of ranked summoners. The positional league endpoints provide a paginated list of all summoners in a Tier + Division + Position (e.g. all ranked Diamond II Top laners). Alternatively, Canisback on the Discord currently hosts a list of league IDs that you can use to pull summoners from the leagues/{leagueId} endpoints. These lists are provided for free to the community for use, and may go down or stop being updated at any time.
    https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries

Helpful tutorial for this
- pulling specific data from API https://www.dataquest.io/blog/python-api-tutorial/ 

List of public API's for playing around another time
- https://github.com/public-apis/public-apis

Other (unused so far) tutorials:
- Writing & reading Json files https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/ 



## Get summoner IDs & names by region, tier & division https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries

In [472]:
# Block: [get summoner ID]
# gets all the summonerName, summonerID, wins & losses for all tiers & divisions in a region and puts them in a summonerID dataframe (todo:) that is connected to a dictionary of tiers and divisions.
# currently uses test dataset tier_list2 & division_list2


# importing tools
import requests
import pandas as pd
import json
import numpy as np
import itertools

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# temp api key location
apikey = 'RGAPI-853eea41-c5c5-4ef7-83a6-9855143e7b98'

# set variables set
region = 'euw1'
queue_type = 'RANKED_SOLO_5x5' 

# create variables that will be looped
tier_list = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
division_list = ['I', 'II', 'III', 'IV']

tier_list2 = ['CHALLENGER'] # testing with these
division_list2 = ['I', 'II'] # testing with these

# iterate through the variables to create the dataset
for tier_counter in tier_list2:      # iterate through the tiers
    for division_counter in division_list2: # iterate through the divisions
        summoner_list = [] # create an empty summoner_list for each tier & division combination. data from more pages will go into the existing list
        print(tier_counter, division_counter)
        page_counter = 1
        keep_going = True
        while keep_going == True:
            url_id = ('https://' + region + '.api.riotgames.com/lol/league-exp/v4/entries/' + queue_type + '/' + str(tier_counter) + '/' + str(division_counter) + '?page=' + str(page_counter) + '&api_key=' + apikey) # create the URL  
            print (url_id)
            response_code = requests.get(url_id) # access the url
            print(str(response_code.status_code)) # response_code is an object, so have to change the type of reponse_code to a string with str(response_code)
            if response_code.status_code == 200: 
                print('if response code')
                response_json = response_code.json() # put the response in a json format
                if bool(response_json):    ##? I think it makes more sense to pull just a little bit of text, no need to put everything in and then run the check, just put in a little bit and if that works keep going. > looking for code efficiency
                    print('if bool')
                    # grab summonerId, summonerName, wins, & losses and append them to a new list summoner_list
                    ## !!! need to append each dataframe to the right division_tier combination dictionary?
                    for summoner_counter in response_json:
                        summoner_list.append([summoner_counter['summonerId'], summoner_counter['summonerName'], summoner_counter['wins'], summoner_counter['losses']])
                        # jprint (summoner_list)
                        summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses']) # create and fill a dataframe from the list summoner_list
                        # summoner_df.head(10)    #displaying the first 10 results of the dataframe
                        # summoner_df.tail(10)    #displaying the last 10 results of the dataframe
                        summoner_dict = summoner_df.to_dict() # converting the dataframe into a dictionary
                        # summoner_dict.keys() # returns a list of all the available keys in the dictionary
                        # summoner_dict.values() # returns a list of all the available values in the dictionary
                    print ('page ', page_counter, 'done')
                    page_counter = page_counter+1
                else:
                    print('else bool')
                    keep_going = False
                    print('page_counter: ', page_counter)
                    break
            else:  # else if there is no data, exit this loop
                print('else response code')
                print('page_counter: ', page_counter)
                break
        
# challenger has 1 page of data
# grandmaster has 4 pages of data

# summoner_df.head(10)    #displaying the first 10 results of the dataframe
summoner_df.tail(10)    #displaying the last 10 results of the dataframe
# will print the challenger I list cuz that one is filled last

CHALLENGER I
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key=RGAPI-853eea41-c5c5-4ef7-83a6-9855143e7b98
200
if response code
if bool
page  1 done
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=2&api_key=RGAPI-853eea41-c5c5-4ef7-83a6-9855143e7b98
200
if response code
if bool
page  2 done
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=3&api_key=RGAPI-853eea41-c5c5-4ef7-83a6-9855143e7b98
200
if response code
else bool
page_counter:  3
CHALLENGER II
https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/II?page=1&api_key=RGAPI-853eea41-c5c5-4ef7-83a6-9855143e7b98
400
else response code
page_counter:  1


,summonerId,summonerName,wins,losses
248,M4SXp1xpyOPloPZ_uuRElVZDJffY8uHaGUW8cl9ImNU3lP4,LLS Emil,117,88
249,sFDPZOQYKEYRZ-_JgMNcOPy7VWuN2z76VdOLFwChDGs6fyE7,Joïnze,107,73
250,ERfKaYhH914vk52KK_uAdkVWxSJDIBOc0lJLUK5c6TNfGHuG,Sinerio,92,83
251,20xSqo16q2FMpkvupb7IJOPgTS34SNzdKqxqLk5FtjSldc0,Jug Kίng,70,48
252,Hb2H8ibHMzzrlyHFgr7bXbz8bJ3qp4NQgugp1z-r8gSLPlg,Mike Wazοwski,50,34
253,kS5Uy7YaZXpxuh6zgvv-KDdb73KRHtUNo0TxoemJg8D5HTE,Sebekx,127,112
254,VkNwjcdl5yLRaAIpFwQ_r7atCPaHDDTGY8j81kP10ChF7QMu,ψτροφήψ,119,103
255,Ge0DEzqDfa3dYbcGJWN3DPGn0y7I0kYPljHG6S3R1PrClf8,LDLC Manaty,72,66
256,nfDdxGmMEYJvwQ0Iup7Wa_0OOAcTTdxPgpmJhsJK3_wLfJ9h,FΙGHT FΙGHT,106,91
257,kW23wtKQGDMdiM1I9tDh0kqTWXEPZS8N1nkNw3QKns_8OAY,it me sharp,102,91


In [ ]:
# remaining to do for above block

    * [ ] create a dataframe that tracks the last page with data for each tier/division
    * [ ] when done put the dataframe in a dict where the key is the tier+division


## Get account ID by summoner ID or name https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerName

In [475]:
# Block: [get account ID]
# gets the account ID using the summoner ID and adds them to the existing summonerID dataframe

# importing tools
import requests
import pandas as pd
import json
import numpy as np
import itertools

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# temp api key location
apikey = 'RGAPI-853eea41-c5c5-4ef7-83a6-9855143e7b98'

# create an empty account list
account_list = []

for accountId_counter in summoner_df['summonerId']:
    #print(summoner_df['summonerId'].head(2)) # print the top 2 values of the column summonerId
    url_id = 'https://' + region + '.api.riotgames.com/lol/summoner/v4/summoners/' + accountId_counter + '?api_key=' + apikey # create the URL 
    # print ('url_id', url_id)
    response_code_account = requests.get(url_id) # access the url
    # print(str(response_code_account.status_code)) # response_code is an object, so have to change the type of reponse_code to a string with str(response_code)
    if response_code_account.status_code == 200: 
        # print('if response code')
        response_json_account = response_code_account.json() # put the response in a json format
        if bool(response_json_account):    ##? I think it makes more sense to pull just a little bit of text, no need to put everything in and then run the check, just put in a little bit and if that works keep going. > looking for code efficiency
            # print('if bool')
            response_json_account = response_code_account.json() # put the response in a json format
            # print('response_json_account', response_json_account) # print the response body
            # print('id=', response_json_account['id']) # print the summonerId ('id' in the response body) from the json response
            account_list.append([response_json_account['id'], response_json_account['accountId']]) # append the summonerId and accountId to the account_list list
            # print('counter=', accountId_counter, 'account_list=', account_list)
        else:
            print('else bool')
    else:
        print('else response code', str(response_code_account.status_code)) # response_code is an object, so have to change the type of reponse_code to a string with str(response_code)

print ('done')
print (account_list)

# append to the existing summoner_df dataframe as a new column at position 0, named "accountId", with the list account_list, do not allow columns with the same name.
summoner_df.insert (0, "accountId", account_list, allow_duplicates = False)
print (summoner_df)

 code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
else response code 429
done
[['g2YD33k6-PJP1GXbOomLnunrGgnh43arV_knNEoKSUJk_zM', 'Bg4Ejd_O72GTvMx8IBMyOignD-1eMifVxacJDcXYjvUUZQ'], ['p5yXs9T4RbRCLVMbKQYEIPYvf6SpLnuV06W4lLk4uplyDVs', 'nnRoPXZ-f4rxwFUtOfElmwnmczB8kjPjPZLcn2H9AEy3nw'], ['PKWTXuIqlW-sVxu9giyen6sKtQEGhqMthdOwb2KXKhAMyc4', 'bLa61l1IQywQN8bZ_kD_3cgJyzTqJS4xxByoxaQ8vlnUHg'], ['RQwkFLRLJAA39aI6qru1PE6NIZgztRPU7hR0PLp1Vq09ApQ', 'ccak3NKbEk8r8sVVRNl2y8kRFRpQSgJVXUIL2iGDtcCDOqg'], ['1PnvjdQ8fp9RrXg9pdJMGb00Del3rqwjFqZwHDr6lnzCR18', 'K71CJD8vIGKUzfPA3bUrMb8Y3FueCSNr7oSzAUBKSiUMGcE'], ['MGV5ICWr6QKalToGZ5p-fykdB6I5hXPwi5oig89iuMy6vfgD', 'nNjqpA0y0EiqLA2L0_rfHRoFC7RV4zRHNayHTIDi2gLH_4vTRairGA8u'], ['Ni0KtYINGti4Xvna202thV4UJgq98

ValueError: Length of values (182) does not match length of index (258)